## Terraforming Mars Cost-Benefit Analysis
<br>
Terraforming Mars is a competitive board game developed by Jacob Fryxelius and published by FryxGames in 2016. Players take on the role of corporations tasked with making Mars habitable through the raising of the oxygen level, temperature, and ocean coverage, as well as sundry other projects to progress humanity.

Currently sitting at #4 on BoardGameGeek's ranked board game list, Terraforming Mars has found broad popularity among strategy board gamers in large part due to its variety and complexity. With 208 unique cards and 12 distinct corporations, players are put into an endless assortment of scenarios. This incredible depth means that players must often intuit their next move rather than deduce it from values and probabilities.

In a bid to add more deduction back into the equation, this analysis aims to take every card in Terraforming Mars and boil it down to a single value in credits (the game's main resource) for simple value-comparison.

Let's get started.

In [11]:
import numpy as np
from statistics import median
import pandas as pd
import seaborn as sns
import csv
import ast

np.set_printoptions(threshold=np.inf)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)


In [124]:
# turn games list txt into csv
with open('Games - 6154.txt') as db:
    game_text = db.read()
game_list = ast.literal_eval(game_text)
    
with open('games_list.csv', 'w') as games_list:
    fields = ['awards', 'corporations', 'draft', 'expansions', 'generations', 'map', 
              'milestones', 'players', 'scores', 'timestamp', 'wgt', 'country', 'colonies', 'rank', 'key']
    output_writer = csv.DictWriter(games_list, fieldnames=fields)
    
    output_writer.writeheader()
    for item in game_list:
        output_writer.writerow(item)

The base_cards database, populated from a hand-entered csv, follows a couple of conventions:
* When a resource is written in uppercase it refers to that resource's production, whereas when written in lowercase it refers to the resource iself.
* All nouns are written in the singular in order to simplify pandas referencing.

The all_games database is a collection of stats from over 12,000 individually logged games downloaded from Simeon Simeonov's excellent [Terraforming Mars website](https://ssimeonoff.github.io/).

In [128]:
# create databases
base_cards = pd.read_csv('TM_base_project_list.csv')
all_games = pd.read_csv('games_list.csv')
base_games = all_games[all_games.expansions == "['CORPORATE']"]

display(base_cards.head(10))

,awards,corporations,draft,expansions,generations,map,milestones,players,scores,timestamp,wgt,country,colonies,rank,key
1,"['LANDLORD', 'BANKER', 'SCIENTIST']","['INVENTRIX', 'ECOLINE', 'THARSIS REPUBLIC', 'TERACTOR', 'INTERPLAN. CINEMATICS']",YES,['CORPORATE'],9,THARSIS,"['MAYOR', 'GARDENER', 'BUILDER']",5,"['55', '60', '59', '55', '50']",NaN,NaN,NaN,NaN,NaN,-LNUHaRC5v7KNHPCHPmU
7,"['THERMALIST', 'MINER']","['UNMI', 'THARSIS REPUBLIC']",NO,['CORPORATE'],13,THARSIS,"['TERRAFORMER', 'GARDENER', 'BUILDER']",2,"['102', '111']",NaN,NaN,NaN,NaN,NaN,-LNUwtqPgLdu_LEQ2bmS
8,"['BANKER', 'SCIENTIST', 'MINER']","['VIRON', 'SATURN SYSTEMS']",NO,['CORPORATE'],14,THARSIS,"['TERRAFORMER', 'MAYOR', 'GARDENER']",2,"['87', '124']",NaN,NaN,NaN,NaN,NaN,-LNUx5vODkdr4LOWb8gR
9,"['BANKER', 'THERMALIST', 'MINER']","['INVENTRIX', 'TERACTOR']",NO,['CORPORATE'],13,THARSIS,"['TERRAFORMER', 'GARDENER', 'BUILDER']",2,"['74', '109']",NaN,NaN,NaN,NaN,NaN,-LNUxIXiV5Ml4wdWq4ed
10,"['LANDLORD', 'THERMALIST', 'MINER']","['THORGATE', 'ECOLINE']",NO,['CORPORATE'],14,THARSIS,"['TERRAFORMER', 'MAYOR', 'GARDENER']",2,"['119', '110']",NaN,NaN,NaN,NaN,NaN,-LNUxWS8oA1dPsqI6tlW
11,"['CELEBRITY', 'INDUSTRIALIST', 'ESTATE DEALER']","['HELION', 'ECOLINE', 'SATURN SYSTEMS', 'INTERPLAN. CINEMATICS']",NO,['CORPORATE'],8,ELYSIUM,"['SPECIALIST', 'ECOLOGIST', 'LEGEND']",4,"['58', '59', '46', '54']",NaN,NaN,NaN,NaN,NaN,-LNV1cGHRwA2PJ6wjvhI
12,"['CELEBRITY', 'INDUSTRIALIST', 'ESTATE DEALER']","['PHOBOLOG', 'UNMI', 'THARSIS REPUBLIC', 'MANUTECH']",NO,['CORPORATE'],9,ELYSIUM,"['GENERALIST', 'ECOLOGIST', 'LEGEND']",4,"['61', '71', '57', '55']",NaN,NaN,NaN,NaN,NaN,-LNV1wF1ThB3esH_Ly3Z
13,"['CELEBRITY', 'DESERT SETTLER', 'BENEFACTOR']","['CREDICOR', 'MINING GUILD', 'VIRON', 'TERACTOR']",NO,['CORPORATE'],11,ELYSIUM,"['ECOLOGIST', 'TYCOON', 'LEGEND']",4,"['83', '79', '71', '74']",NaN,NaN,NaN,NaN,NaN,-LNV2BaHse9gQVOtLEME
15,"['INDUSTRIALIST', 'DESERT SETTLER', 'BENEFACTOR']","['SATURN SYSTEMS', 'THARSIS REPUBLIC', 'UNMI']",NO,['CORPORATE'],9,ELYSIUM,"['SPECIALIST', 'ECOLOGIST', 'TYCOON']",3,"['82', '77', '66']",NaN,NaN,NaN,NaN,NaN,-LNVAyy_-Wyg6XRy85W6
17,"['MAGNATE', 'EXCENTRIC', 'CONTRACTOR']","['THORGATE', 'INTERPLAN. CINEMATICS', 'HELION', 'CREDICOR']",NO,['CORPORATE'],9,HELLAS,"['TACTICIAN', 'POLAR EXPLORER', 'ENERGIZER']",4,"['55', '56', '52', '60']",NaN,NaN,NaN,NaN,NaN,-LNVBCCxju3M-R_mDdkD


In [74]:
# determine generation averages per player count
lastgen2p = int(round(base_games[base_games.players == 2].generations.mean()))
lastgen3p = int(round(base_games[base_games.players == 3].generations.mean()))
lastgen4p = int(round(base_games[base_games.players == 4].generations.mean()))
lastgen5p = int(round(base_games[base_games.players == 5].generations.mean()))

lastgens = {
            'lastgen2p': int(round(base_games[base_games.players == 2].generations.mean())),
            'lastgen3p': int(round(base_games[base_games.players == 3].generations.mean())),
            'lastgen4p': int(round(base_games[base_games.players == 4].generations.mean())),
            'lastgen5p': int(round(base_games[base_games.players == 5].generations.mean()))
           }

print(f'''
Average length of game in generations:
2p: {lastgen2p}
3p: {lastgen3p}
4p: {lastgen4p}
5p: {lastgen5p}
''')

# average game length split into quarters with the median generation taken from each quarter
# X players, quarter N, median generation
gen_quarters = {
               '2pq1gen':round(lastgen2p/4/2),
               '2pq2gen':round(lastgen2p/4/2 + lastgen2p/4),
               '2pq3gen':round(lastgen2p/4/2 + lastgen2p/4*2),
               '2pq4gen':round(lastgen2p/4/2 + lastgen2p/4*3),
               '3pq1gen':round(lastgen3p/4/2),
               '3pq2gen':round(lastgen3p/4/2 + lastgen3p/4),
               '3pq3gen':round(lastgen3p/4/2 + lastgen3p/4*2),
               '3pq4gen':round(lastgen3p/4/2 + lastgen3p/4*3),
               '4pq1gen':round(lastgen4p/4/2),
               '4pq2gen':round(lastgen4p/4/2 + lastgen4p/4),
               '4pq3gen':round(lastgen4p/4/2 + lastgen4p/4*2),
               '4pq4gen':round(lastgen4p/4/2 + lastgen4p/4*3),
               '5pq1gen':round(lastgen5p/4/2),
               '5pq2gen':round(lastgen5p/4/2 + lastgen5p/4),
               '5pq3gen':round(lastgen5p/4/2 + lastgen5p/4*2),
               '5pq4gen':round(lastgen5p/4/2 + lastgen5p/4*3)
              }
keys2pgen = ['2pq1gen', '2pq2gen', '2pq3gen', '2pq4gen']
keys3pgen = ['3pq1gen', '3pq2gen', '3pq3gen', '3pq4gen']
keys4pgen = ['4pq1gen', '4pq2gen', '4pq3gen', '4pq4gen']
keys5pgen = ['5pq1gen', '5pq2gen', '5pq3gen', '5pq4gen']

print(f'''
Generation quarters:
2p: {str([gen_quarters.get(key) for key in keys2pgen]).strip('[]')}
3p: {str([gen_quarters.get(key) for key in keys3pgen]).strip('[]')}
4p: {str([gen_quarters.get(key) for key in keys4pgen]).strip('[]')}
5p: {str([gen_quarters.get(key) for key in keys5pgen]).strip('[]')}
''')



Average length of game in generations:
2p: 13
3p: 11
4p: 10
5p: 9


Generation quarters:
2p: 2, 5, 8, 11
3p: 1, 4, 7, 10
4p: 1, 4, 6, 9
5p: 1, 3, 6, 8



In [49]:
# variable definitions

# generation-based variables
remgen = 0
victory_point = 0
TR = victory_point + remgen

# standard projects
temp = 14
ocean = 18
greenery = 23
ENERGY = 11
city = 25 + remgen

# resources
steel = 2
titanium = 3
plant = 2.875
heat = 1.75
energy = 3
draw = 4

In [77]:
# define credits per victory point
def credits_per_vp(generation, player_count):
    for players in lastgens:
        if str(player_count) in players:
            lastgen = lastgens[players]
    income = lastgen - generation
    temp_ppc = 1/(temp - income) # ppc = points per credit
    ocean_ppc = 1/(ocean - income)
    greenery_ppc = 2/(greenery - income)
    ppc = (temp_ppc + ocean_ppc + greenery_ppc)/4
    credits_per_vp = 1/ppc
    return round(credits_per_vp)

# numbers of credits per victory point for each player count and median generation
# X players, quarter N, credits per point
gen_credits_per_vp = {'2pq1cpp':credits_per_vp(2, 2),
                      '2pq2cpp':credits_per_vp(5, 2),
                      '2pq3cpp':credits_per_vp(8, 2),
                      '2pq4cpp':credits_per_vp(11, 2),
                      '3pq1cpp':credits_per_vp(1, 3),
                      '3pq2cpp':credits_per_vp(4, 3),
                      '3pq3cpp':credits_per_vp(7, 3),
                      '3pq4cpp':credits_per_vp(10, 3),
                      '4pq1cpp':credits_per_vp(1, 4),
                      '4pq2cpp':credits_per_vp(4, 4),
                      '4pq3cpp':credits_per_vp(6, 4),
                      '4pq4cpp':credits_per_vp(9, 4),
                      '5pq1cpp':credits_per_vp(1, 5),
                      '5pq2cpp':credits_per_vp(3, 5),
                      '5pq3cpp':credits_per_vp(6, 5),
                      '5pq4cpp':credits_per_vp(8, 5)
                     }

keys2pcpp = ['2pq1cpp', '2pq2cpp', '2pq3cpp', '2pq4cpp']
keys3pcpp = ['3pq1cpp', '3pq2cpp', '3pq3cpp', '3pq4cpp']
keys4pcpp = ['4pq1cpp', '4pq2cpp', '4pq3cpp', '4pq4cpp']
keys5pcpp = ['5pq1cpp', '5pq2cpp', '5pq3cpp', '5pq4cpp']

print(f'''
Generation credits per point per quarter:
   q1, q2, q3, q4
2p: {str([gen_credits_per_vp.get(key) for key in keys2pcpp]).strip('[]')}
3p: {str([gen_credits_per_vp.get(key) for key in keys3pcpp]).strip('[]')}
4p: {str([gen_credits_per_vp.get(key) for key in keys4pcpp]).strip('[]')}
5p: {str([gen_credits_per_vp.get(key) for key in keys5pcpp]).strip('[]')}
''')


Generation credits per point per quarter:
   q1, q2, q3, q4
2p: 6, 10, 13, 17
3p: 8, 11, 14, 18
4p: 9, 12, 14, 18
5p: 10, 12, 16, 18



In [122]:
# Food Factory test
def food_factory(generation, player_count):
    for players in lastgens:
        if str(player_count) in players:
            lastgen = lastgens[players]
    income = lastgen - generation
    #determine quarter of game
    quarters_list = []
    for gen in gen_quarters:
        if gen[0] == str(player_count):
            quarters_list.append(gen_quarters[gen])
    absolute_difference_function = lambda list_value : abs(list_value - generation)
    for i in range(4):
        if quarters_list[i] == min(quarters_list, key=absolute_difference_function):
            quarter = i+1
    #determine credits per vp
    for cpp in gen_credits_per_vp:
        if cpp[0] == str(player_count) and cpp[3] == str(quarter):
            vp = gen_credits_per_vp[cpp]
    value = 0
    value -= 12 #card cost
    value -= round(plant*(income+1))
    value += round(4*income)
    value += vp
    return value
    
food_factory(1, 2)    

4